In [ ]:
from tensorflow.keras.layers import Attention, Dropout, GroupNormalization, BatchNormalization, Flatten, Dense, Conv1D, MaxPooling1D,InputLayer, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import sklearn.linear_model
from PIL import Image
import tensorflow as tf
from google.colab import drive
from PIL import Image
drive.mount('/content/drive/')
DIR = 'drive/MyDrive/SHTEM24PAPAYA/'

Mounted at /content/drive/


In [ ]:
def padding(small, img_back, hi = 1/5, vi = 1/5):
  small_dims = small.shape
  big_dims = img_back.shape
  horiz_pad = big_dims[1]-small_dims[1]
  vert_pad = big_dims[0]-small_dims[0]
  left_pad = int(np.round(horiz_pad*hi,0))
  up_pad = int(np.round(vert_pad*vi,0))
  padded = np.zeros_like(img_back)
  padded[up_pad:up_pad+small_dims[0], left_pad:left_pad+small_dims[1]] = small
  return padded

In [ ]:
x = np.load(DIR+'350x.npy')
y = np.load(DIR+'350y.npy')

In [ ]:
#WITHOUT BACTHNORM
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import EfficientNetB0


def build_efficientnetb0_model(k, f, input_shape=(512, 512, 3)):

    base_model = EfficientNetB0 (weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    encoder_output = base_model.output

    up1 = layers.UpSampling2D((2, 2))(encoder_output)
    conv1 = layers.Conv2D(f, (k, k), activation='relu', padding='same')(up1)
    conv1 = layers.Conv2D(f*2, (k, k), activation='relu', padding='same')(conv1)
    up2 = layers.UpSampling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(f*2, (k, k), activation='relu', padding='same')(up2)
    conv2 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(conv2)
    up3 = layers.UpSampling2D((2, 2))(conv2)
    conv3 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(up3)
    up4 = layers.UpSampling2D((2, 2))(conv3)
    conv4 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(up4)
    up5 = layers.UpSampling2D((2, 2))(conv4)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up5)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

for k in [3,4,5]:
    for f in [2,4,8]:
        efficientnetb0Model = build_efficientnetb0_model(k,f)

        efficientnetb0Model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.BinaryIoU(
            target_class_ids=(0, 1), threshold=0.5, name=None, dtype=None
            )])

        efficientnetb0Model.summary()

        trainnum = int(len(x)*0.9)
        e= efficientnetb0Model.fit(x[:trainnum], y[:trainnum], validation_data=(x[trainnum:], y[trainnum:]), epochs=5, batch_size=32)

        pd.DataFrame(e.history).to_csv(DIR+'Histories/blakeTestEfficientnetb0_k'+str(k)+'_f'+str(f)+'_350_nobatchnorm.csv')

In [ ]:
#WITH BATCHNORM
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import EfficientNetB0


def build_efficientnetb0_model(k,f, input_shape=(512, 512, 3)):

    base_model = EfficientNetB0 (weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    encoder_output = base_model.output

    up1 = layers.UpSampling2D((2, 2))(encoder_output)
    conv1 = layers.Conv2D(f, (k, k), activation='relu', padding='same')(up1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Conv2D(f*2, (k, k), activation='relu', padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    up2 = layers.UpSampling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(up2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    up3 = layers.UpSampling2D((2, 2))(conv2)
    conv3 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(up3)
    conv3 = layers.BatchNormalization()(conv3)
    up4 = layers.UpSampling2D((2, 2))(conv3)
    conv4 = layers.Conv2D(f*4, (k, k), activation='relu', padding='same')(up4)
    conv4 = layers.BatchNormalization()(conv4)
    up5 = layers.UpSampling2D((2, 2))(conv4)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up5)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

for k in [3,4,5]:
    for f in [2,4,8]:
        efficientnetb0Model = build_efficientnetb0_model(k,f)

        efficientnetb0Model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.BinaryIoU(
            target_class_ids=(0, 1), threshold=0.5, name=None, dtype=None
            )])

        efficientnetb0Model.summary()

        trainnum = int(len(x)*0.9)
        g= efficientnetb0Model.fit(x[:trainnum], y[:trainnum], validation_data=(x[trainnum:], y[trainnum:]), epochs=5, batch_size=32)

        pd.DataFrame(g.history).to_csv(DIR+'Histories/efficientnetb0_k'+str(k)+'_f'+str(f)+'_350_batchnorm.csv')

16705208/16705208 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 512, 512, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 512, 512, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     